In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import StackingClassifier

In [45]:
train_df = pd.read_csv("output.csv")
test_df = pd.read_csv("output.csv")

In [46]:
train_df.head(3)

,id,Ложь,Аггравация,Экстраверсия,Спонтанность,Агрессивность,Ригидность,Интроверсия,Сензитивность,Тревожность,Лабильность,Эмоциональное_состояние_опрошенного,Результат
0,0,4.0,4.0,4.0,2.0,3.0,7.0,5.0,3.0,6.0,4.0,отлично,0
1,1,8.0,7.0,6.0,6.0,5.0,5.0,5.0,5.0,6.0,6.0,удовлетворительно,0
2,2,6.0,7.0,4.0,4.0,3.0,4.0,5.0,3.0,3.0,5.0,отлично,0


In [47]:
test_df.head(3)

,id,Ложь,Аггравация,Экстраверсия,Спонтанность,Агрессивность,Ригидность,Интроверсия,Сензитивность,Тревожность,Лабильность,Эмоциональное_состояние_опрошенного,Результат
0,0,4.0,4.0,4.0,2.0,3.0,7.0,5.0,3.0,6.0,4.0,отлично,0
1,1,8.0,7.0,6.0,6.0,5.0,5.0,5.0,5.0,6.0,6.0,удовлетворительно,0
2,2,6.0,7.0,4.0,4.0,3.0,4.0,5.0,3.0,3.0,5.0,отлично,0


In [48]:
cat_cols = [
    "Эмоциональное_состояние_опрошенного"]
num_cols = ['id','Ложь','Аггравация','Экстраверсия','Спонтанность','Агрессивность',
            'Ригидность','Интроверсия','Сензитивность','Тревожность','Лабильность']

In [49]:
# заполняем пробелы
for column in num_cols:
    train_df[column].fillna(train_df[column].mean(), inplace=True)
for column in cat_cols:
    train_df[column].fillna(train_df[column].mode()[0], inplace=True)

In [50]:
test_train_df = train_df.drop(cat_cols, axis=1)
test_train_df = test_train_df.drop(["id", "Результат"], axis=1)

In [51]:
matrix = test_train_df.corr().abs() # создаем матрицу корреляций

threshold = 0.7 # порог можно поставить любой,
                # для примера возьмем 0.7, однако, можно поэкспериментировать
                # с разными значениями

upper = matrix.where(np.triu(np.ones(matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
catboost_df = test_train_df.drop(to_drop, axis=1)
catboost_df.corr().style.background_gradient(cmap='coolwarm')

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Сензитивность,Тревожность
Ложь,1.000000,0.459512,0.301452,-0.195658,-0.070195,-0.058203,0.044908
Аггравация,0.459512,1.000000,0.120989,-0.506348,-0.474630,0.060545,0.327010
Экстраверсия,0.301452,0.120989,1.000000,-0.346922,-0.461113,0.050978,-0.250754
Спонтанность,-0.195658,-0.506348,-0.346922,1.000000,0.463529,0.307470,-0.093908
Ригидность,-0.070195,-0.474630,-0.461113,0.463529,1.000000,-0.226134,0.145402
Сензитивность,-0.058203,0.060545,0.050978,0.307470,-0.226134,1.000000,0.542523
Тревожность,0.044908,0.327010,-0.250754,-0.093908,0.145402,0.542523,1.000000


In [52]:
# I prepared a function for reviewing the dataset.
def Df_info(df):
    print("########### Head ###########")
    print(df.head())
    print("########### Tail ###########")
    print(df.tail())
    print("########### Shape ###########")
    print(df.shape)
    print("########### Info ###########")
    print(df.info())
    print("########### Info ###########")
    print(df.columns)
    print("########### Quantiles ###########")
    print(df.describe().T)
    print("########### NA ###########")
    print(df.isnull().sum())

In [53]:
catboost_df[cat_cols] = train_df[cat_cols]

# Df_info(catboost_df)

In [54]:
Df_info(catboost_df)

########### Head ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Сензитивность  \
0   4.0         4.0           4.0           2.0         7.0            3.0   
1   8.0         7.0           6.0           6.0         5.0            5.0   
2   6.0         7.0           4.0           4.0         4.0            3.0   
3   7.0         5.0           5.0           5.0         6.0            5.0   
4   4.0         4.0           7.0           5.0         5.0            3.0   

   Тревожность Эмоциональное_состояние_опрошенного  
0          6.0                             отлично  
1          6.0                   удовлетворительно  
2          3.0                             отлично  
3          8.0                 неудовлетворительно  
4          3.0                             отлично  
########### Tail ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Сензитивность  \
5   5.0         2.0           8.0           8.0         7.0            4.0   

In [55]:
for column in num_cols:
    test_df[column].fillna(test_df[column].mean(), inplace=True)
for column in cat_cols[:-1]:
    test_df[column].fillna(test_df[column].mode()[0], inplace=True)

In [56]:
catboost_df.shape

(10, 8)

In [57]:
catboost_df["Результат"] = train_df["Результат"]

In [58]:
catboost_df

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Сензитивность,Тревожность,Эмоциональное_состояние_опрошенного,Результат
0,4.0,4.0,4.0,2.0,7.0,3.0,6.0,отлично,0
1,8.0,7.0,6.0,6.0,5.0,5.0,6.0,удовлетворительно,0
2,6.0,7.0,4.0,4.0,4.0,3.0,3.0,отлично,0
3,7.0,5.0,5.0,5.0,6.0,5.0,8.0,неудовлетворительно,1
4,4.0,4.0,7.0,5.0,5.0,3.0,3.0,отлично,0
5,5.0,2.0,8.0,8.0,7.0,4.0,3.0,удовлетворительно,1
6,6.0,2.0,1.0,9.0,9.0,3.0,3.0,неудовлетворительно,1
7,3.0,2.0,5.0,6.0,4.0,5.0,3.0,удовлетворительно,1
8,5.0,6.0,6.0,4.0,6.0,4.0,4.0,отлично,0
9,1.0,4.0,1.0,9.0,7.0,5.0,7.0,неудовлетворительно,1


In [59]:
test_df = test_df[catboost_df.columns[:-1]]
test_df.shape

(10, 8)

In [60]:
test_df

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Сензитивность,Тревожность,Эмоциональное_состояние_опрошенного
0,4.0,4.0,4.0,2.0,7.0,3.0,6.0,отлично
1,8.0,7.0,6.0,6.0,5.0,5.0,6.0,удовлетворительно
2,6.0,7.0,4.0,4.0,4.0,3.0,3.0,отлично
3,7.0,5.0,5.0,5.0,6.0,5.0,8.0,неудовлетворительно
4,4.0,4.0,7.0,5.0,5.0,3.0,3.0,отлично
5,5.0,2.0,8.0,8.0,7.0,4.0,3.0,удовлетворительно
6,6.0,2.0,1.0,9.0,9.0,3.0,3.0,неудовлетворительно
7,3.0,2.0,5.0,6.0,4.0,5.0,3.0,удовлетворительно
8,5.0,6.0,6.0,4.0,6.0,4.0,4.0,отлично
9,1.0,4.0,1.0,9.0,7.0,5.0,7.0,неудовлетворительно


In [61]:
catboost_df[cat_cols] = catboost_df[cat_cols].astype("str")
test_df[cat_cols] = test_df[cat_cols].astype("str")

In [63]:
X = catboost_df.drop(["Результат"], axis=1)
y = catboost_df["Результат"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [64]:
Df_info(catboost_df)

########### Head ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Сензитивность  \
0   4.0         4.0           4.0           2.0         7.0            3.0   
1   8.0         7.0           6.0           6.0         5.0            5.0   
2   6.0         7.0           4.0           4.0         4.0            3.0   
3   7.0         5.0           5.0           5.0         6.0            5.0   
4   4.0         4.0           7.0           5.0         5.0            3.0   

   Тревожность Эмоциональное_состояние_опрошенного  Результат  
0          6.0                             отлично          0  
1          6.0                   удовлетворительно          0  
2          3.0                             отлично          0  
3          8.0                 неудовлетворительно          1  
4          3.0                             отлично          0  
########### Tail ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Сензитивность  \
5   5.0    

In [65]:
model = CatBoostClassifier(iterations=1000,
                           depth=6,
                           task_type="GPU",
                           devices='0:1')
model.fit(x_train, y_train, cat_features=cat_cols)

Learning rate set to 0.043965
0:	learn: 0.6533117	total: 23.8ms	remaining: 23.8s
1:	learn: 0.6163968	total: 36.3ms	remaining: 18.1s
2:	learn: 0.5861485	total: 55.4ms	remaining: 18.4s
3:	learn: 0.5576935	total: 75.2ms	remaining: 18.7s
4:	learn: 0.5285062	total: 89.1ms	remaining: 17.7s
5:	learn: 0.4993393	total: 102ms	remaining: 16.9s
6:	learn: 0.4730496	total: 114ms	remaining: 16.1s
7:	learn: 0.4525819	total: 127ms	remaining: 15.8s
8:	learn: 0.4262013	total: 139ms	remaining: 15.3s
9:	learn: 0.4119799	total: 160ms	remaining: 15.8s
10:	learn: 0.3940654	total: 171ms	remaining: 15.4s
11:	learn: 0.3773997	total: 187ms	remaining: 15.4s
12:	learn: 0.3626139	total: 201ms	remaining: 15.3s
13:	learn: 0.3480323	total: 216ms	remaining: 15.2s
14:	learn: 0.3347285	total: 234ms	remaining: 15.3s
15:	learn: 0.3216209	total: 248ms	remaining: 15.2s
16:	learn: 0.3110740	total: 266ms	remaining: 15.4s
17:	learn: 0.2998879	total: 283ms	remaining: 15.4s
18:	learn: 0.2869225	total: 297ms	remaining: 15.3s
19:	le

In [66]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [70]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

FileNotFoundError: [Errno 2] No such file or directory: 'cluster_weights.xlsx'

In [68]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape

(2, 2)

In [69]:
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

NameError: name 'weights_dict' is not defined

In [ ]:
test_df_2.pivot(index="id", columns="date", values="start_cluster").head(3)

date,month_4,month_5,month_6
id,,,
200000,{α},{α},None
200001,{α},{α},None
200002,{other},{other},None


In [ ]:
test_df_2["start_cluster"] = catboost_df["start_cluster"].mode()[0]
test_df_2["start_cluster"] = test_df_2["start_cluster"].astype("category")

In [ ]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [ ]:
sample_submission_df.shape

(100000, 18)

In [ ]:
sample_submission_df.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [ ]:
test_df['date'] =test_df_2["date"]

last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

In [ ]:
test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [ ]:
test_pred_proba_df.shape

(100000, 17)

In [ ]:
test_pred_proba_df.head(2)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.005147,0.027717,0.013145,0.012869,0.001848,0.000013,0.000135,5.911749e-05,0.003954,0.001776,0.005569,0.000184,0.000578,4.105285e-07,0.001079,0.925926,3.800488e-07
1,0.003690,0.608799,0.000392,0.000631,0.000457,0.000020,0.000010,2.794296e-07,0.000437,0.002219,0.000267,0.000027,0.000131,1.003349e-08,0.000320,0.382546,5.216470e-05


In [ ]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("output.csv", index=False)